In [ ]:
# Decorators::

def make_pretty(func):
    def inner():
        print("I got decorated")
        func()
        print("I got decorated - 2")
    return inner

def ordinary():
    print("I am ordinary")

# ordinary()

# make_pretty() is a decorator
ordinary = make_pretty(ordinary)
ordinary()

In [ ]:
def make_pretty(func):
    print("I Got decorated.")
    func()

def ordinary():
    print("I am ordinary")

make_pretty(ordinary)

In [ ]:
# Decorators::

def make_pretty(func):
    def inner():
        print("I got decorated")#FuncStarted
        func()
        print("I am ending decoration")
        #Func Ended
    return inner

# # make_pretty() is a decorator
# ordinary = make_pretty(ordinary)
# ordinary()

@make_pretty
def ordinary():
    print("I am ordinary")

ordinary()

In [9]:
# Usage of Decorators 

def percent(func):
    print("Hi")
    def inner(*args, **kwargs):
        print("%" * 30)
        func(*args, **kwargs)
        print("%" * 30)
    return inner

# printer = percent(printer)

@percent
def printer(msg):
    print(msg)

@percent
def printer_new(msg, msg1):
    print(msg, " : ", msg1)

print("Calling Decorated Function")
# print(type(printer))
printer("Hello")
printer_new("Hello", "Trainees")

Hi
Hi
Calling Decorated Function
<class 'function'>
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Hello
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Hello  :  Trainees
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


In [ ]:
class ESRequestRetry(object):
  """
    Decorator Class for retrying ES requests for certain number of times.

    Args:
      func(function): Function to be decorated

    Returns:
      Function instance of the decorated function.
  """

  def __init__(self, retry=constants.CONFIG["elasticsearch"][
    "connection_retries"], sleep_seconds=constants.CONFIG["elasticsearch"][
    "retry_interval"]):
    self.retry = retry
    self.sleep_seconds = sleep_seconds

  def percent(self, func):

    @wraps(func)
    def inner(*args, **kwargs):
      """
      Wrapper method for decorator
      """
      LOG.debug("Function = %s, args = %s, kwargs = %s"
                % (func.__name__, args, kwargs))

      for retry_counter in range(self.retry):
        try:
          return func(*args, **kwargs)
        except Exception as ex:                # pylint: disable = broad-except
          LOG.error(str(ex), exc_info=True)

        if retry_counter == self.retry - 1:
          LOG.error("Maximum retries reached. Exiting...")
          raise ex

        LOG.debug("Waiting for '%s' more seconds..." % self.sleep_seconds)
        time.sleep(self.sleep_seconds)

    return inner


@decorators.ESRequestRetry()
def update_share_record_in_es(query_body, share_uuid, es_client):
  """
  Update Share record in ES

  Args:
    share_uuid(str): Share UUID
    query_body(dict): query body
    es_client(obj): ES client object

  Returns:
    None
  """
  index = constants.FILE_SERVERS_INDEX
  LOG.debug(query_body)
  try:
    response = es_client.update(index=index, doc_type="doc", id=share_uuid,
                                body=query_body,
                                request_timeout=constants.CONFIG[
                                  "elasticsearch"]["query_timeout"])
    LOG.debug(response)
  except es_exceptions.ConnectionError as ce:
    LOG.error(str(ce), exc_info=True)
    raise ce
  except es_exceptions.ConnectionTimeout as ct:
    LOG.error(str(ct), exc_info=True)
    raise ct
  except es_exceptions.RequestError as re:
    LOG.error(str(re), exc_info=True)
    raise re
  except es_exceptions.NotFoundError as nfe:
    LOG.error(str(nfe), exc_info=True)
    raise Exception("Share with uuid %s does not exist" % share_uuid)
  except es_exceptions.TransportError as te:
    LOG.error(str(te), exc_info=True)
    raise te

In [ ]:
# Decorating functions with parameters
# parameters of the nested inner() function inside the decorator is same as the parameters of functions it decorates.
# Taking this into account, now we can make general decorators that work with any number of parameter.

def smart_divide(func):
   def inner(a,b):
      print("I am going to divide",a,"and",b)
      if b == 0:
         print("Whoops! cannot divide")
         return

      return func(a,b)
   return inner

@smart_divide
def divide(a,b):
    return a/b

ret = divide(2,5)
print(ret)
divide(2,0)

In [ ]:
# Nested Decorators 

def star(func):
    def inner(*args, **kwargs):
        print("*" * 30)
        func(*args, **kwargs)
        print("*" * 30)
    return inner

def percent(func):
    def inner(*args, **kwargs):
        print("%" * 30)
        func(*args, **kwargs)
        print("%" * 30)
    return inner

# printer = star(percent(printer))
@star
@percent
def printer(msg, msg1="Hi"):
    print(msg, " : ", msg1)
printer("Hello")

In [ ]:
def smart_divide(func):
   def inner(a):
      print("I am going to divide",a)
      if a == 0:
         print("Whoops! cannot divide")
         return
    
   return inner

@smart_divide
def divide(a,b):
    print("Test")

print(divide(2))
divide(2)